## Demonstration of GA4GH schema repository with SRA sequencing Experiment Metadata

NCBI SRA publishes attributes of sequencing runs and experiments in the Cloud via BigQuery and Athena. Details are provided [here](https://www.ncbi.nlm.nih.gov/sra/docs/sra-cloud-based-metadata-table/)

The data in the table above derives from BioProject and BioSample. Besides a set of columns available for all studies in the table, the attributes column allows key-value pairs to be provided for custom attributes for a given study (BioPeoject).

This notebook explores how a schema registry can assist in describing the custom metadata for specific studies. With this information it is possible to reconcile attributes across studies.

For a set of six NCBI and EBI studies a schema was generated to list the project/study specific attributes.

The studies are as follows:
* sra_PRJEB10573 - [Characterization of RBP9 and RBP10, two developmentally regulated RNA-binding proteins in Trypanosoma brucei](https://www.ebi.ac.uk/ena/browser/view/PRJEB14125)
* sra_PRJEB37886 - [COVID-19 Genomics UK (COG-UK) consortium](https://www.ebi.ac.uk/ena/browser/view/PRJEB37886)
* sra_PRJEB1985 - [Whole genome re-sequencing of a single Bos taurus animal for SNP discovery](https://www.ebi.ac.uk/ena/browser/view/PRJEB1985)
* sra_phs001554 - [dbGaP Colon Cancer study (BioProject PRJNA437032)](https://www.ncbi.nlm.nih.gov/bioproject/437032)
* sra_scr_icac - [dbGaP Childhood Asthma study (BioProject PRJNA837686)](https://www.ncbi.nlm.nih.gov/bioproject/PRJNA837686)
* sra_scr_udn_v5 - [dbGaP Undiagnosed Disease Network study (BioProject PRJNA350185)](https://www.ncbi.nlm.nih.gov/bioproject/PRJNA350185)

These are a largely unrelated set of studies selected for the purpose of this demonstration. Others may be added.


#### Imports

In [34]:
import requests;

import json;

import xml.etree.ElementTree as ET

def prettyprint(a_dict):
    print(json.dumps(a_dict, indent=3))

def printline(char="_"):
    print(char*80)
    

In [88]:
base = "http://localhost:8080"

In [79]:
url = f"{base}/namespaces"
response = requests.get(url)
namespaces = response.json()['namespaces']
for namespace in namespaces:
    print( namespace['namespace_name'])

dbGaP
dataconnect-demo
expt-metadata


### Fetch all schemas for the expt-metadata namespace

In [89]:
namespaces = ["expt-metadata"]
expt_schemas = {}
for namespace in namespaces:
    printline("=")
    print(f"getting schemas for namespace {namespace}")
    url = f"{base}/schemas/{namespace}/"
    response = requests.get(url)
    schemas = response.json()['schemas']
    printline()
    for schema in schemas:
        print( )
        url = f"{base}/schemas/{namespace}/{schema['schema_name']}/versions/v2"
        print(f"fetching schema {schema['schema_name']}")
        response = requests.get(url)
        response_schema = response.json()
        expt_schemas[schema['schema_name']] = response_schema
        #prettyprint( schema)
        printline()

getting schemas for namespace expt-metadata
________________________________________________________________________________

fetching schema sra_PRJEB10573
________________________________________________________________________________

fetching schema sra_PRJEB1985
________________________________________________________________________________

fetching schema sra_PRJEB37886
________________________________________________________________________________

fetching schema sra_phs001554
________________________________________________________________________________

fetching schema sra_scr_icac
________________________________________________________________________________

fetching schema sra_scr_udn_v5
________________________________________________________________________________


In [74]:
import pandas as pd

tab_list = []
include_single_values = False
for study_name, study_schema in expt_schemas.items():
    tab_row = {"study":study_name}
    for pname, prop_dict in study_schema['properties'].items():
        if include_single_values and prop_dict['unique_count'] == 1:
            tab_row[pname] = prop_dict['enums'][0]
        else:
            tab_row[pname] = prop_dict['$unique_count']
    tab_list.append(tab_row)
summary_df = pd.DataFrame.from_dict(tab_list)
summary_df.set_index('study', inplace=True)
summary_df        

,bases,broker_name_sam,bytes,cell_line_sam,cell_type_sam_ss_dpl37,common_name_sam,ena_first_public_sam,ena_last_update_sam,experimental_factor__over_expression_dazl_exp,experimental_factor__over_expression_nanos3_exp,...,gap_accession_sam,is_tumor_sam,molecular_data_type_sam,phs,sex_calc,study_design_sam,study_disease_sam,study_name_sam,subject_is_affected_sam,submitted_subject_id_sam
study,,,,,,,,,,,,,,,,,,,,,
sra_PRJEB10573,9,1.0,9,1.0,1.0,1.0,1.0,3.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sra_PRJEB1985,49,NaN,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sra_PRJEB37886,7511,3.0,7511,NaN,NaN,NaN,465.0,270.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sra_phs001554,2892,NaN,2892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,5.0,2.0,1.0,2.0,1.0,2.0,2892.0
sra_scr_icac,1035,NaN,1035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,NaN,1.0,2.0,1.0,NaN,1.0,NaN,1035.0
sra_scr_udn_v5,9847,NaN,10028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,2.0,1.0,2.0,1.0,2.0,1.0,3.0,6469.0


### Schema comparison across studies

In the following table...
* Each column represents a different study
* Each row represents a different attribute key
* The numbers indicate the number of unique values the given attribute takes for each study
* A value of 0 indicates that the the attribute is not present in the schema for that study

In [85]:
pd.set_option('display.max_rows', None)
attribute_summary_df = summary_df.transpose()
attribute_summary_df.fillna(0, inplace=True)
for scname in expt_schemas.keys():
    attribute_summary_df[scname] = attribute_summary_df[scname].astype(int)
attribute_summary_df

study,sra_PRJEB10573,sra_PRJEB1985,sra_PRJEB37886,sra_phs001554,sra_scr_icac,sra_scr_udn_v5
bases,9,49,7511,2892,1035,9847
broker_name_sam,1,0,3,0,0,0
bytes,9,49,7511,2892,1035,10028
cell_line_sam,1,0,0,0,0,0
cell_type_sam_ss_dpl37,1,0,0,0,0,0
common_name_sam,1,0,0,0,0,0
ena_first_public_sam,1,0,465,0,0,0
ena_last_update_sam,3,0,270,0,0,0
experimental_factor__over_expression_dazl_exp,2,0,0,0,0,0
experimental_factor__over_expression_nanos3_exp,2,0,0,0,0,0


### Show the table as a heat map
A quick overview of which attributes are used in which studies can be seen in the following heat map.

In [86]:
for scname in expt_schemas.keys():
    attribute_summary_df.loc[attribute_summary_df[scname] > 0 , scname] = 1
attribute_summary_df
attribute_summary_df.style.background_gradient(cmap ='viridis').set_properties(**{'font-size': '1px'}) 

study,sra_PRJEB10573,sra_PRJEB1985,sra_PRJEB37886,sra_phs001554,sra_scr_icac,sra_scr_udn_v5
bases,1,1,1,1,1,1
broker_name_sam,1,0,1,0,0,0
bytes,1,1,1,1,1,1
cell_line_sam,1,0,0,0,0,0
cell_type_sam_ss_dpl37,1,0,0,0,0,0
common_name_sam,1,0,0,0,0,0
ena_first_public_sam,1,0,1,0,0,0
ena_last_update_sam,1,0,1,0,0,0
experimental_factor__over_expression_dazl_exp,1,0,0,0,0,0
experimental_factor__over_expression_nanos3_exp,1,0,0,0,0,0


### Stable SRA metadata schema

In addition to the custom set of attributes covered above there are a common set of columns/sttributes for every SRA sequencing run. 

The schema for the common columns is described at https://www.ncbi.nlm.nih.gov/sra/docs/sra-cloud-based-metadata-table/

Note that all of the common attributes are populated for all studies. Investigating this would be a useful addition to identifying the experiment/run metadata for a given study.

In [81]:
namespace ="dataconnect-demo"        
schema_name ="nih_sra_datastore.sra.metadata"        
url = f"{base}/schemas/{namespace}/{schema_name}/versions/v2"
print("fetching schema")
response = requests.get(url)
response_schema = response.json()
prettyprint(response_schema)

fetching schema
{
   "$id": "",
   "description": "Metadata Table (sra.metadata) contains information about the run and biological samples. Metadata Table (sra.metadata) contains information about the run and biological samples. The biological sample data is stored in two different columns.  See Record (array) column that you need to use the command UNNEST to query. See https://www.ncbi.nlm.nih.gov/sra/docs/sra-cloud-based-metadata-table/",
   "$schema": "http://json-schema.org/draft-07/schema",
   "properties": {
      "acc": {
         "type": "string",
         "description": "SRA Run accession in the form of SRR######## (ERR or DRR for INSDC partners)"
      },
      "assay_type": {
         "type": "string",
         "description": "Type of library (i.e. AMPLICON, RNA-Seq, WGS, etc)",
         "oneOf": [
            {
               "title": "RNA-Seq"
            },
            {
               "title": "WGS"
            },
            {
               "title": "EST"
            }